In [3]:
from plexsim.models import *
n = 128
g = nx.grid_graph([n, n])
# g = nx.grid_2d_graph(n, n, periodic = 1)

settings = dict(graph = g,\
                updateType = 'async',\
#                 sampleSize = .25,\
                alpha = 4,\
#                 mu = .2,\
#                 nu = .02,\
#                 kappa = .001,\
#                 beta = .2,\
#                 threshold = .01,\
#                 p = .1,\
               t = 0.5, 
)

nx.Graph()
m = 4 * 2
rules = nx.complete_graph(m)
et = {(u, v): -1 for u, v in rules.edges()}
print(et)
nx.set_edge_attributes(rules, name = 'weight',  values = et)
# rules.add_edge(m-1, 0, weight = 0)

# rules.add_edge(0, 1)
# rules.add_edge(1, 2)
# rules.add_edge(2, 0, weight = 0)

eigen = 0.

gg = nx.complete_graph(m//2)
for u, v in gg.edges():
    rules[u][v]['weight'] = 0
gg = nx.cycle_graph(m // 2)
for u, v in gg.edges():
    rules[u][v]['weight'] = 1
    
gg = nx.complete_graph(m//2)
gg = nx.relabel_nodes(gg, {i : (m-1) - i for i in gg.nodes() })
for u, v in gg.edges():
    rules[u][v]['weight'] = 0
    
gg = nx.cycle_graph(m // 2)
gg = nx.relabel_nodes(gg, {i : (m-1) - i for i in gg.nodes() })
for u, v in gg.edges():
    rules[u][v]['weight'] = 1
# for node in rules:
#     rules.add_edge(node, node, weight = eigen)
# rules[0][4]['weight'] = -1
# print(nx.node_link_data(rules))
# rules.add_edge(2, 0)

ti = time.time()
ag = np.arange(m).tolist()
models = dict(
    Potts_val_memless  = Potts(agentStates = ag,\
#                   rules = rules,\
          **settings),\
#     Pott_val_mem = Potts(agentStates = ag, rules = rules,\
#                  memorySize = 1,\
#                  memento = 0, \
#                  **settings),
#     Ising = Ising(\
#           **settings),\
#     Bornholdt = Bornholdt(\
#               **settings),\
#     RBN = RBN(**settings),\
#     SIRS = SIRS(\
#         **settings),\
#    CCA = CCA(\
#         agentStates = np.arange(0,4).tolist(),\
#         **settings),\
#    Percolation = Percolation(\
#                **settings),\
   
)
print(f"Settup time was {time.time() - ti}")
# models.get("Bornholdt").sampleSize = 1
if m := models.get("SIRS"):
    m.states = 0
    m.states[m.sampleNodes(1)[0,0]] = 1
print("starting sims")

# mi = list(models.values())
# for i in mi:
#     print(i.memory.shape, i.memento)
# assert 0
T = 500
import time
start = time.time()
results = np.zeros((len(models), T, g.number_of_nodes()))
for idx, m in enumerate(models.values()):
#     m.states = m.agentStates[-1]
    m.reset()
    results[idx] = m.simulate(T)
r = results.mean(-1).flatten()
results = results.reshape(-1, T, n, n)

# print(results.shape)
# results = results[-1, T,:, :, 0]
print(f'Simulation took {time.time() - start}')

import matplotlib.pyplot as plt
from ipywidgets import interact, IntSlider
import matplotlib as mpl
from mpl_toolkits.axes_grid1 import make_axes_locatable as mal
%matplotlib widget
ts = IntSlider(min = 0, max = T - 1, value = 0)

columns, rows = divmod(len(models), 2)
# columns = columns + 1 if not columns else columns
# rows = rows + 1 if not rows else rows
# fig, ax = plt.subplots()
# ax.plot(r)
# fig.show()
fig, ax = plt.subplots(rows + 1, columns + 1, figsize = (7,7))
fig.subplots_adjust(wspace = .12, hspace = .0001)
# ax = np.array([ax])

#setup figure/
hs  = []
pad = .01
size = '5%'
for idx, ( (modelName, m), axi) in enumerate(zip(models.items(), ax.flat)):
    div = mal(axi)
    cbr = div.append_axes('bottom', size = size, pad = pad)
#     div = mal(cbr)
#     cbr = div.append_axes('right', size = size, pad = pad)
    AS = m.agentStates
    NS = len(AS) 
    cmap = mpl.colors.LinearSegmentedColormap.from_list(\
          None, plt.cm.twilight(np.linspace(0, 1, NS + 1)[:NS]), NS)
    
    h = axi.imshow(results[idx, 0], cmap = cmap, vmin = AS[0],\
                  vmax = AS[-1])
    axi.set_title(modelName)
    axi.axis("off")
    tmp = fig.colorbar(h, cax = cbr, ticks = AS,\
                       orientation = 'horizontal')
    if modelName == 'SIRS':
        tmp.set_ticklabels("Sus. Inf. Rec.".split())
    hs.append(h)
mainax = fig.add_subplot(111, frameon = 0,\
                        xticks = [], yticks = [])
[axi.axis('off') for axi in ax.flat[rows * columns:]]
mainax.set_xlabel(f'T = 0')
# fig.subplots_adjust(left = 0, right = 1, bottom = 0, top = 1)
fig.tight_layout()
@interact(t = ts)
def update(t):
    for idx, h in enumerate(hs):
        h.set_data(results[idx, t])
#         h.autoscale()
    mainax.set_xlabel(f'T = {t}')
    fig.canvas.flush_events()
    fig.canvas.draw()
fig.show()

{(0, 1): -1, (0, 2): -1, (0, 3): -1, (0, 4): -1, (0, 5): -1, (0, 6): -1, (0, 7): -1, (1, 2): -1, (1, 3): -1, (1, 4): -1, (1, 5): -1, (1, 6): -1, (1, 7): -1, (2, 3): -1, (2, 4): -1, (2, 5): -1, (2, 6): -1, (2, 7): -1, (3, 4): -1, (3, 5): -1, (3, 6): -1, (3, 7): -1, (4, 5): -1, (4, 6): -1, (4, 7): -1, (5, 6): -1, (5, 7): -1, (6, 7): -1}
Settup time was 0.6604783535003662
starting sims
Simulation took 5.621544599533081


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

interactive(children=(IntSlider(value=0, description='t', max=499), Output()), _dom_classes=('widget-interact'…

array([[ 12,   5, 136,  42,  68,  67,  64,  48, 155,  16,  66,  81,  96,
        159, 135, 189, 213, 185, 101,  43,  99, 174, 222, 117, 218, 125,
         70, 195, 239,  40,  45, 201, 191, 249, 165, 252, 242, 118,  55,
         22,  33,  86, 149, 247, 133, 192, 188,  97,  51,  88,  59,  36,
        116,  89, 212, 127,  94, 202,  24, 241, 168,  50, 167,  20,  28,
        132, 210,  82,   4,  34, 157, 235, 141,  47, 115, 230, 147,  79,
         83, 234, 194,   6, 224, 198, 215,   3,   9,  78, 251, 206, 253,
        121,  25,  76, 237, 119, 197, 231,  92,  56, 184,  17, 181, 173,
         44, 182, 142, 178,  14, 107, 131, 205, 148, 166, 109, 236, 140,
        175, 227, 204,  41, 223, 138,  72, 123,  60,  10,  31, 156, 137,
         80,  95, 193, 187, 203, 228,  21, 152,  37, 108,  87,  26, 163,
         84,   0, 100, 180, 158,  13, 216,  61, 229,  57, 211, 143, 169,
         63, 225, 160, 207, 233,  29,  54,  65, 122, 112,   8,  49, 171,
        250,  98, 111,  58,  32, 220,  93, 106,  27

In [ ]:
# fig, ax = plt.subplots()
# # ax.plot(results[0].reshape(T, -1).T)
# ax.plot(results[0].reshape(T, -1).mean(1), label = 'valued')
# ax.plot(results[1].reshape(T,-1).mean(1), label = 'original')
# ax.legend()
# fig.show()

print(np.linspace(0, 255, m.nStates, dtype = int))
# rules = nx.circular_ladder_graph(10)
from Utils.plotting import addGraphPretty as agp
fig, ax = plt.subplots()
agp(rules, ax = ax, with_labels = 1, circle = dict(radius = .1),\
   annotate = dict(fontsize = 190))
fig.show()

for u, v in rules.edges():
    print(u, v, rules[u][v])

In [ ]:


def generate_random_3Dgraph(n_nodes, radius, seed=None):
    if seed is not None:
        random.seed(seed)
    
    # Generate a dict of positions
    pos = {i: (random.uniform(0, 1), random.uniform(0, 1), random.uniform(0, 1)) for i in range(n_nodes)}
    
    # Create random 3D network
    G = nx.random_geometric_graph(n_nodes, radius, pos=pos)
    return G


def network_plot_3D(G, angle, save=False):
    # Get node positions
    pos = nx.get_node_attributes(G, 'pos')

    # Get number of nodes
    n = G.number_of_nodes()
    # Get the maximum number of edges adjacent to a single node
    edge_max = max([G.degree(i) for i in range(n)])
    # Define color range proportional to number of edges adjacent to a single node
    colors = [plt.cm.plasma(G.degree(i)/edge_max) for i in range(n)] 
    # 3D network plot
    contexts = 'seaborn-poster bmh'.split()
    with plt.style.context(contexts):

        fig = plt.figure(figsize=(10,7))
        ax = Axes3D(fig)

        # Loop on the pos dictionary to extract the x,y,z coordinates of each node
        for key, value in pos.items():
            xi = value[0]
            yi = value[1]
            zi = value[2]

            # Scatter plot
            ax.scatter(xi, yi, zi, c=colors[key], s=20+20*G.degree(key), edgecolors='k', alpha=0.7)

        # Loop on the list of edges to get the x,y,z, coordinates of the connected nodes
        # Those two points are the extrema of the line to be plotted
        for i,j in enumerate(G.edges()):
            x = np.array((pos[j[0]][0], pos[j[1]][0]))
            y = np.array((pos[j[0]][1], pos[j[1]][1]))
            z = np.array((pos[j[0]][2], pos[j[1]][2]))

        # Plot the connecting lines
            ax.plot(x, y, z, color = 'black', alpha = 1)

    # Set the initial view
    ax.view_init(20, angle)
    # Hide the axes
    ax.set_axis_off()
    fig.subplots_adjust(top = 1, bottom = 0,\
                       left = 0, right = 1)
    fig.tight_layout()
    ax.margins(0, 0, 0)
    if save:
         fig.savefig(save +".png", bbox_inches = 'tight',\
                    pad_inches = 0)
         plt.close('all')
    else:
          fig.show()

    return
n=200
G = generate_random_3Dgraph(n_nodes=n, radius=0.25, seed=1)
network_plot_3D(G, 20, save= "top_plot")

g = generate_random_3Dgraph(n_nodes = n, radius = .1,\
                            seed = 1)
network_plot_3D(g, 20, save = 'bottom_plot')